In [1]:
import os
import pandas as pd
import requests
from xml.etree import ElementTree as ET
from datetime import datetime
import geojson


class EarthquakeAnalyzer:
    def __init__(self, download_path: str = "./data5"):
        self.download_path = download_path
        if not os.path.exists(download_path):
            os.mkdir(download_path)
    
    def query_period(self, start_year: int, start_month: int, end_year: int, end_month: int) -> list:
        """Download earthquake data for a specific period"""
        downloaded_files = []
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
       
        now = datetime.now()
        current_year = now.year
        current_month = now.month
        
        for year in range(start_year, end_year + 1):
            start_m = start_month if year == start_year else 1
            end_m = end_month if year == end_year else 12
            
            for month in range(start_m, end_m + 1):
                file_name = os.path.join(self.download_path, f"{year}{month:02}.xml")
                
                
                is_current_month = (year == current_year and month == current_month)
                
                if os.path.exists(file_name) and not is_current_month:
                    print(f"✓ Already exists: {year}-{month:02}")
                    downloaded_files.append(file_name)
                    continue
                
                url = f"http://udim.koeri.boun.edu.tr/zeqmap/xmlt/{year}{month:02}.xml"
                
                try:
                    response = requests.get(url, headers=headers, timeout=10)
                    response.raise_for_status()
                    
                    if len(response.content) < 100:
                        print(f"⚠️  {year}-{month:02}: Empty response")
                        continue
                    
                    with open(file_name, "wb") as f:
                        f.write(response.content)
                    
                    downloaded_files.append(file_name)
                    status = "Downloaded" if not os.path.exists(file_name) else "Refreshed"
                    print(f"✓ {status}: {year}-{month:02}")
                    
                except Exception as e:
                    print(f"✗ Error {year}-{month:02}: {e}")
        
        return downloaded_files
        
    def extract_data(self, file_paths: list) -> dict:
        """Analyze earthquake data from XML files"""
        earthquakes = []
        
        for file_path in file_paths:
            try:
                tree = ET.parse(file_path)
                root = tree.getroot()
                
                
                for event in root.findall("earhquake"):
                    magnitude = float(event.get("mag", 0))
                    latitude = float(event.get("lat", 0))
                    longitude = float(event.get("lng", 0))
                    depth = float(event.get("Depth", 0))
                    timestamp = event.get("name", "")
                    location = event.get("lokasyon", "").strip()
                    
                    if magnitude > 0:
                        earthquakes.append({
                            "timestamp": timestamp,
                            "location": location,
                            "magnitude": magnitude,
                            "latitude": latitude,
                            "longitude": longitude,
                            "depth": depth
                        })
            except Exception as e:
                print(f"Error parsing {file_path}: {e}")
        
        
        earthquakes = pd.DataFrame.from_dict(earthquakes)
        return earthquakes

In [2]:
def extract_cities(df: pd.DataFrame) -> pd.DataFrame:
    """Extract city names from location column (text in parentheses at the end)"""
    def get_city(location):
        if '(' in location and ')' in location:
            start = location.rfind('(')
            end = location.rfind(')')
            if start < end:
                return location[start+1:end].strip()
        return None
    
    df['city'] = df['location'].apply(get_city)
    return df

In [3]:
analyzer = EarthquakeAnalyzer(download_path="./data5")
files = analyzer.query_period(start_year=2025, start_month=10, end_year=2025, end_month=11)
data = analyzer.extract_data(files)
#print(f"Downloaded: {len(files)} files")
#print(f"Total earthquakes: {results['total_earthquakes']}")
data = extract_cities(data)

✓ Already exists: 2025-10
✓ Refreshed: 2025-11


In [4]:
with open('faults/gem_active_faults.geojson', encoding='utf-8') as f:
    gj = geojson.load(f)
features = gj['features'][0]

In [5]:
def filter_features_by_bounds(features, min_lat, max_lat, min_lng, max_lng):
    """Filter GeoJSON features by coordinate bounding box"""
    filtered = []
    
    for feature in features:
        try:
            coords = feature['geometry']['coordinates']
            
            if feature['geometry']['type'] == 'Point':
                lng, lat = coords[0], coords[1]
                if min_lat <= lat <= max_lat and min_lng <= lng <= max_lng:
                    filtered.append(feature)
            
            elif feature['geometry']['type'] in ['LineString', 'MultiPoint']:
                for coord in coords:
                    lng, lat = coord[0], coord[1]
                    if min_lat <= lat <= max_lat and min_lng <= lng <= max_lng:
                        filtered.append(feature)
                        break
            
            elif feature['geometry']['type'] == 'Polygon':
                for ring in coords:
                    for coord in ring:
                        lng, lat = coord[0], coord[1]
                        if min_lat <= lat <= max_lat and min_lng <= lng <= max_lng:
                            filtered.append(feature)
                            break
        
        except (ValueError, IndexError, TypeError):
            # Skip features with invalid geometry
            continue
    
    return filtered


with open('faults/gem_active_faults.geojson', encoding='utf-8') as f:
    gj = geojson.load(f)

filtered_features = filter_features_by_bounds(
    gj['features'],
    min_lat=36.0,   # Southern limit
    max_lat=42.0,   # Northern limit
    min_lng=26.0,   # Western limit
    max_lng=45.0    # Eastern limit
)

print(f"Total features: {len(gj['features'])}")
print(f"Filtered features: {len(filtered_features)}")

Total features: 16195
Filtered features: 840


In [6]:
import pandas as pd

features_list = []
for feature in filtered_features:
    row = feature.get('properties', {}).copy()
    row['geometry_type'] = feature['geometry']['type']
    
    # Add coordinates
    coords = feature['geometry']['coordinates']
    if feature['geometry']['type'] == 'Point':
        row['longitude'] = coords[0]
        row['latitude'] = coords[1]
    elif feature['geometry']['type'] in ['LineString', 'MultiPoint']:
        row['coordinates'] = coords
    elif feature['geometry']['type'] == 'Polygon':
        row['coordinates'] = coords
    
    features_list.append(row)

features_df = pd.DataFrame(features_list)
features_df

,average_dip,average_rake,catalog_id,catalog_name,epistemic_quality,lower_seis_depth,net_slip_rate,slip_type,upper_seis_depth,geometry_type,coordinates,activity_confidence,shortening_rate,strike_slip_rate
0,"(45.0,35.0,55.0)","(-65,-80.0,-50.0)",ME_BGCS001,EMME,1,"(15.0,,)","(0.2,0.1,0.4)",Sinistral-Normal,"(0.0,,)",LineString,"[[26.347992, 41.844493], [26.234177, 41.8739],...",NaN,NaN,NaN
1,"(47.5,40.0,55.0)","(-75,-90.0,-60.0)",ME_BGCS006,EMME,1,"(15.0,,)","(0.2,0.1,0.4)",Normal,"(1.0,,)",LineString,"[[26.176791, 41.711203], [26.123651, 41.72388]...",NaN,NaN,NaN
2,"(62.5,50.0,75.0)","(-80,-90.0,-70.0)",ME_BGCS007,EMME,1,"(15.0,,)","(0.2,0.1,0.4)",Normal,"(1.0,,)",LineString,"[[26.591698, 41.834274], [26.566001, 41.858706...",NaN,NaN,NaN
3,"(60.0,45.0,75.0)","(-105,-130.0,-80.0)",ME_GRCS160,EMME,1,"(14.0,,)","(0.6,0.4,0.7)",Normal,"(0.0,,)",LineString,"[[26.129644, 40.847688], [26.075288, 40.845883...",NaN,NaN,NaN
4,"(62.5,45.0,80.0)","(-105,-120.0,-90.0)",ME_GRCS170,EMME,1,"(14.0,,)","(0.3,0.3,0.3)",Normal,"(0.0,,)",LineString,"[[26.121717, 41.542607], [26.165257, 41.551238...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,"(75.0,65,85)","(-125,-110,-140)",EUR_TRCS912,SHARE,NaN,"(15.0,,)","(0.8,0.5,1.0)",Dextral-Normal,"(0.0,,)",LineString,"[[27.519627, 37.838987], [27.504157, 37.831566...",NaN,NaN,NaN
836,"(62.5,50,75)","(-85,-100,-70)",EUR_TRCS913,SHARE,NaN,"(14.5,,)","(1.8,0.5,3.0)",Normal,"(0.0,,)",LineString,"[[27.391798, 36.936121], [27.471934, 36.951685...",NaN,NaN,NaN
837,"(72.5,55,90)","(22,180,-135)",EUR_TRCS996,SHARE,NaN,"(19.0,,)","(27.7,26.5,28.8881)",Sinistral,"(0.0,,)",LineString,"[[28.026022, 40.80224], [27.881072, 40.811477]...",NaN,NaN,NaN
838,"(70.0,50,90)","(35,10,60)",EUR_TRCS999,SHARE,NaN,"(25.0,,)","(1.7,0.4,3.0)",Sinistral-Reverse,"(2.0,,)",LineString,"[[34.879748, 35.865749], [34.942292, 35.944967...",NaN,NaN,NaN


In [ ]:
from scipy.spatial.distance import cdist
import numpy as np

def find_closest_fault(earthquake_lat, earthquake_lng, faults_df):
    """Find closest fault line to an earthquake"""
    try:

        fault_coords = []
        for coords in faults_df['coordinates']:
            if isinstance(coords, list) and len(coords) > 0:

                if isinstance(coords[0], list):
                    fault_coords.append(coords[0][:2])
                else:
                    fault_coords.append(coords[:2])
        
        if not fault_coords:
            return None, float('inf')
        

        distances = cdist(
            [[earthquake_lng, earthquake_lat]], 
            fault_coords
        )[0]
        
        closest_idx = np.argmin(distances)
        return closest_idx, distances[closest_idx]
    
    except Exception:
        return None, float('inf')


data['closest_fault_idx'] = data.apply(
    lambda row: find_closest_fault(
        row['latitude'], 
        row['longitude'], 
        features_df
    )[0],
    axis=1
)

data['distance_to_fault'] = data.apply(
    lambda row: find_closest_fault(
        row['latitude'], 
        row['longitude'], 
        features_df
    )[1],
    axis=1
)


data = data.merge(
    features_df.reset_index().rename(columns={'index': 'closest_fault_idx'}),
    on='closest_fault_idx',
    how='left',

)

print(data[['timestamp', 'magnitude', 'city', 'distance_to_fault', 'catalog_id']].head())

             timestamp  magnitude       city  distance_to_fault   catalog_id
0  2025.10.01 00:03:01        1.8  BALIKESIR           0.142560   ME_TRCS210
1  2025.10.01 00:06:38        1.9    KUTAHYA           0.157171   ME_TRCS209
2  2025.10.01 00:22:32        2.1  SANLIURFA           0.170300  EUR_TRCS128
3  2025.10.01 00:44:40        1.7     ANKARA           0.054702  EUR_TRCS237
4  2025.10.01 00:52:50        1.1  BALIKESIR           0.120546  EUR_TRCS210


In [9]:
data

,timestamp,location,magnitude,latitude,longitude,depth,city,closest_fault_idx,distance_to_fault,average_dip_eq,...,epistemic_quality_fault,lower_seis_depth_fault,net_slip_rate_fault,slip_type_fault,upper_seis_depth_fault,geometry_type_fault,coordinates_fault,activity_confidence_fault,shortening_rate_fault,strike_slip_rate_fault
0,2025.10.01 00:03:01,PURSUNLER-SINDIRGI (BALIKESIR),1.8,39.2210,28.2110,7.4,BALIKESIR,187,0.142560,"(50.0,50.0,50.0)",...,1,"(15.0,,)","(5.0,4.9605,5.0246)",Normal,"(1.0,,)",LineString,"[[28.332585, 39.146565], [28.281087, 39.166223...",NaN,NaN,NaN
1,2025.10.01 00:06:38,YEMISLI-SIMAV (KUTAHYA),1.9,39.2535,28.9675,12.0,KUTAHYA,186,0.157171,"(56.0,56.0,56.0)",...,1,"(15.0,,)",None,Normal,"(1.0,,)",LineString,"[[28.90582, 39.108938], [28.865417, 39.115577]...",NaN,NaN,NaN
2,2025.10.01 00:22:32,KARAKAS-BOZOVA (SANLIURFA),2.1,37.2968,38.4728,14.0,SANLIURFA,534,0.170300,"(80.0,80,80)",...,NaN,"(20.0,,)","(1.5,1.0,2.0)",Dextral,"(0.0,,)",LineString,"[[38.642876, 37.305538], [38.538336, 37.339435...",NaN,NaN,NaN
3,2025.10.01 00:44:40,CANKAYA (ANKARA),1.7,39.8723,32.7593,6.9,ANKARA,642,0.054702,"(60.0,60,60)",...,NaN,"(12.0,,)","(0.5,0.0001,1.0)",Sinistral,"(0.0,,)",LineString,"[[32.74757, 39.81887], [32.743088, 39.728205],...",NaN,NaN,NaN
4,2025.10.01 00:52:50,ORMANICI-SINDIRGI (BALIKESIR),1.1,39.1617,28.0863,16.2,BALIKESIR,616,0.120546,"(50.0,50,50)",...,NaN,"(15.0,,)","(5.0,4.9605,5.0246)",Normal,"(0.0,,)",LineString,"[[28.023449, 39.264565], [28.126545, 39.225212...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8751,2025.11.11 17:02:48,YAYLACIK-SINDIRGI (BALIKESIR),1.5,39.1902,28.1735,16.5,BALIKESIR,187,0.164961,"(50.0,50.0,50.0)",...,1,"(15.0,,)","(5.0,4.9605,5.0246)",Normal,"(1.0,,)",LineString,"[[28.332585, 39.146565], [28.281087, 39.166223...",NaN,NaN,NaN
8752,2025.11.11 17:10:52,ORMANICI-SINDIRGI (BALIKESIR),2.2,39.1803,28.1192,16.1,BALIKESIR,616,0.127549,"(50.0,50,50)",...,NaN,"(15.0,,)","(5.0,4.9605,5.0246)",Normal,"(0.0,,)",LineString,"[[28.023449, 39.264565], [28.126545, 39.225212...",NaN,NaN,NaN
8753,2025.11.11 17:15:41,SINANDEDE-SINDIRGI (BALIKESIR),1.6,39.1805,28.1498,19.2,BALIKESIR,616,0.151761,"(50.0,50,50)",...,NaN,"(15.0,,)","(5.0,4.9605,5.0246)",Normal,"(0.0,,)",LineString,"[[28.023449, 39.264565], [28.126545, 39.225212...",NaN,NaN,NaN
8754,2025.11.11 17:23:08,YAYLACIK-SINDIRGI (BALIKESIR),1.4,39.1932,28.1780,10.1,BALIKESIR,187,0.161466,"(50.0,50.0,50.0)",...,1,"(15.0,,)","(5.0,4.9605,5.0246)",Normal,"(1.0,,)",LineString,"[[28.332585, 39.146565], [28.281087, 39.166223...",NaN,NaN,NaN


✓ Already exists: 2025-10
✓ Refreshed: 2025-11


In [ ]:
data